# Training the Encoder

As mentioned in the report, the encoder used is a ResNet, of varying sizes.

We begin by importing some librarie.

In [1]:
import torch
#import kagglehub
from torchvision.transforms import v2
import os #for loading the data
from torch import nn
from torch.nn import functional as F
import kagglehub

For the data, we define the path everything is been taken out of and which augmentations we're going to use.

In [2]:
path = kagglehub.dataset_download("abdelghaniaaba/wildfire-prediction-dataset")

In [3]:
from utils import create_weak_aug,create_strong_aug,create_valid_transform
weak = create_weak_aug(size = (224,224))
strong = create_strong_aug(size = (224,224))
valid_transform = create_valid_transform(size=(224,224))

In [4]:
from torchvision.datasets.folder import ImageFolder
from datasets import UnlabelledImageFolder,our_loader,labelled_TensorDataset

In [5]:
train_path = os.path.join(path,'train')

In [6]:
from datasets import unlabelled_TensorDataset
unlabelled_set = unlabelled_TensorDataset(name = train_path,transform=weak,target_transform=strong)


Premature end of JPEG file


We defined all the parameters for the training, as in the other notebooks.

In [7]:
num_workers = 2
epochs=100
#Training params and hyperParams
batch_size = 64 
#Optim Parameters
lr = 1e-3
momentum = 0.5
weight_decay = 0.03
nesterov=True

In [8]:
from torch.utils.data import DataLoader
unlabel_loader = DataLoader(unlabelled_set,batch_size=int(batch_size),shuffle=True, pin_memory=True, num_workers=num_workers,persistent_workers=True)


Here we use the loss as defined in https://arxiv.org/abs/2011.00362 .

In [9]:
from utils import info_nce

criterion = info_nce(temperature=0.5)

In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In the <em> encoder.py </em> file, we have the definitions of the encoder, and also the networkd we'll be using to fine-tune it into the labelled data later.

We made functions that create an encoder for the following variations of Resnet: ResNet18, ResNet34 and ResNet50. 
https://arxiv.org/abs/1512.03385

In [11]:
from encoders import resnet18_encode,resnet34_encode,resnet50_encode

In [12]:
from utils import epoch_loop_encode

models = ["resnet"+str(value) for value in [18,34,50]]
creators = [resnet18_encode,resnet34_encode,resnet50_encode]

for i in range(len(models)):
    save_path = models[i]+"_enc_strong"+".pth.tar"
    model = creators[i]()
    model.to(device)
    print("Training ",save_path)
    optimizer = torch.optim.SGD(model.parameters(), lr=lr,momentum=momentum, weight_decay=weight_decay,nesterov=nesterov)

    epoch_loop_encode(model,unlabel_loader,optimizer,criterion,device,epochs,save_path=save_path)

Training  resnet18_enc_strong.pth.tar
[Epoch: 1/100] Training
[Epoch: 1/100] Avg loss: 4.0869

Best model so far. Saving model as model.pth

[Epoch: 2/100] Training
[Epoch: 2/100] Avg loss: 3.9975

Best model so far. Saving model as model.pth

[Epoch: 3/100] Training
[Epoch: 3/100] Avg loss: 3.8900

Best model so far. Saving model as model.pth

[Epoch: 4/100] Training
[Epoch: 4/100] Avg loss: 3.7348

Best model so far. Saving model as model.pth

[Epoch: 5/100] Training
[Epoch: 5/100] Avg loss: 3.6373

Best model so far. Saving model as model.pth

[Epoch: 6/100] Training
[Epoch: 6/100] Avg loss: 3.5645

Best model so far. Saving model as model.pth

[Epoch: 7/100] Training
[Epoch: 7/100] Avg loss: 3.5027

Best model so far. Saving model as model.pth

[Epoch: 8/100] Training
[Epoch: 8/100] Avg loss: 3.4591

Best model so far. Saving model as model.pth

[Epoch: 9/100] Training
[Epoch: 9/100] Avg loss: 3.4131

Best model so far. Saving model as model.pth

[Epoch: 10/100] Training
[Epoch: 10

KeyboardInterrupt: 